In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
import time

# Cache for storing quantum circuits to avoid recreating them
circuit_cache = {}

def create_oracle(values, target_idx, num_qubits):
    cache_key = f'oracle_{target_idx}_{num_qubits}'
    if cache_key in circuit_cache:
        return circuit_cache[cache_key]

    oracle = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        if (target_idx >> i) & 1:
            oracle.x(i)
    
    if num_qubits == 1:
        oracle.h(0)
        oracle.z(0)
        oracle.h(0)
    elif num_qubits > 3:
        mid = num_qubits // 2
        oracle.h(num_qubits - 1)
        oracle.mcx(list(range(mid)), mid)
        oracle.mcx(list(range(mid, num_qubits - 1)), num_qubits - 1)
        oracle.h(num_qubits - 1)
    else:
        oracle.h(num_qubits - 1)
        if num_qubits == 2:
            oracle.cx(0, 1)
        else:
            oracle.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    
    for i in range(num_qubits):
        if (target_idx >> i) & 1:
            oracle.x(i)
    circuit_cache[cache_key] = oracle
    return oracle

def create_diffusion(num_qubits):
    cache_key = f'diffusion_{num_qubits}'
    if cache_key in circuit_cache:
        return circuit_cache[cache_key]

    diffusion = QuantumCircuit(num_qubits + 1)
    for qubit in range(num_qubits):
        diffusion.h(qubit)
    for qubit in range(num_qubits):
        diffusion.x(qubit)
    chunk_size = 3
    for i in range(0, num_qubits - 1, chunk_size):
        control_qubits = list(range(i, min(i + chunk_size, num_qubits - 1)))
        if len(control_qubits) > 0:
            diffusion.h(num_qubits)
            diffusion.mcx(control_qubits, num_qubits)
            diffusion.h(num_qubits)
    for qubit in range(num_qubits):
        diffusion.x(qubit)
    for qubit in range(num_qubits):
        diffusion.h(qubit)
    circuit_cache[cache_key] = diffusion
    return diffusion

def grover_find_min_index(values):
    n = len(values)
    num_bits = max(1, int(np.ceil(np.log2(n))))
    min_idx = np.argmin(values)
    
    qr = QuantumRegister(num_bits + 1, 'q')
    cr = ClassicalRegister(num_bits, 'c')
    circuit = QuantumCircuit(qr, cr)
    
    for i in range(num_bits):
        circuit.h(qr[i])
    
    iterations = int(np.pi/4 * np.sqrt(2**num_bits))
    oracle = create_oracle(values, min_idx, num_bits + 1)
    diffusion = create_diffusion(num_bits)
    
    for _ in range(iterations):
        circuit = circuit.compose(oracle)
        circuit = circuit.compose(diffusion)
    
    for i in range(num_bits):
        circuit.measure(qr[i], cr[i])
    
    backend = Aer.get_backend('aer_simulator')
    result = backend.run(circuit, shots=1000).result()
    counts = result.get_counts()
    max_count_result = max(counts.items(), key=lambda x: x[1])[0]
    return int(max_count_result, 2) % n

def quantum_sort_cluster(cluster_df, sort_column):
    df = cluster_df.copy()
    sorted_indices = []
    values = df[sort_column].tolist()
    remaining_indices = list(range(len(values)))
    
    while remaining_indices:
        remaining_values = [values[i] for i in remaining_indices]
        min_idx = grover_find_min_index(remaining_values)
        actual_idx = remaining_indices[min_idx]
        sorted_indices.append(actual_idx)
        remaining_indices.remove(actual_idx)
    
    return df.iloc[sorted_indices].reset_index(drop=True)

def cluster_based_quantum_sort(input_csv, sort_column, n_clusters=4, output_csv='cluster_sorted.csv'):
    start_time = time.time()
    
    df = pd.read_csv(input_csv)
    df = df.dropna()
    
    if sort_column not in df.columns:
        print(f"Column '{sort_column}' not found.")
        return
    
    print("Original Data:\n", df)
    
    clustering_data = df[[sort_column]]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(clustering_data)
    
    all_sorted = []
    for cluster_id in range(n_clusters):
        cluster_df = df[df['cluster'] == cluster_id].drop(columns=['cluster'])
        print(f"\nSorting Cluster {cluster_id} (size {len(cluster_df)}):")
        sorted_cluster = quantum_sort_cluster(cluster_df, sort_column)
        all_sorted.append(sorted_cluster)
    
    merged_df = pd.concat(all_sorted, ignore_index=True)
    final_sorted_df = merged_df.sort_values(by=sort_column).reset_index(drop=True)
    
    print("\nFinal Sorted Data:")
    print(final_sorted_df)

    end_time = time.time()
    print(f"Total execution time: {end_time - start_time:.2f} seconds")

if __name__ == "__main__":
    circuit_cache.clear()
    cluster_based_quantum_sort('phpB0xrNj.csv', sort_column="f3", n_clusters=20)

Original Data:
           f1      f2      f3      f4      f5      f6      f7      f8      f9  \
0    -0.4394 -0.0930  0.1718  0.4620  0.6226  0.4704  0.3578  0.0478 -0.1184   
1    -0.4348 -0.1198  0.2474  0.4036  0.5026  0.6328  0.4948  0.0338 -0.0520   
2    -0.2330  0.2124  0.5014  0.5222 -0.3422 -0.5840 -0.7168 -0.6342 -0.8614   
3    -0.3808 -0.0096  0.2602  0.2554 -0.4290 -0.6746 -0.6868 -0.6650 -0.8410   
4    -0.3412  0.0946  0.6082  0.6216 -0.1622 -0.3784 -0.4324 -0.4358 -0.4966   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
7331 -0.6842 -0.3280 -0.1984  0.2956  0.8786  0.8948  0.3118  0.1822  0.1012   
7332 -0.5912 -0.2420  0.8174  1.0000  0.4642  0.6428  0.6944  0.3056 -0.3888   
7333 -0.6696 -0.3730  0.1584  0.8910  1.0000  0.9762  0.9762  0.7684  0.4106   
7334 -0.5764 -0.1764  0.5106  0.3742 -0.1670 -0.5858 -0.7882 -0.7224 -0.6330   
7335 -0.6624 -0.3334  0.3666  0.4292 -0.2084 -0.5374 -0.4542 -0.6208 -0.6376   

         f10  ...    f6

In [57]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import time

def classical_sort_cluster(cluster_df, sort_column):
    return cluster_df.sort_values(by=sort_column).reset_index(drop=True)

def classical_cluster_based_sort(input_csv, sort_column, n_clusters=4):
    start_time = time.time()
    
    df = pd.read_csv(input_csv)
    df = df.dropna()
    
    if sort_column not in df.columns:
        print(f"Column '{sort_column}' not found.")
        return
    
    print("Original Data:\n", df)
    
    clustering_data = df[[sort_column]]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(clustering_data)
    
    all_sorted = []
    for cluster_id in range(n_clusters):
        cluster_df = df[df['cluster'] == cluster_id].drop(columns=['cluster'])
        print(f"\nSorting Cluster {cluster_id} (size {len(cluster_df)}):")
        sorted_cluster = classical_sort_cluster(cluster_df, sort_column)
        all_sorted.append(sorted_cluster)
    
    merged_df = pd.concat(all_sorted, ignore_index=True)
    final_sorted_df = merged_df.sort_values(by=sort_column).reset_index(drop=True)
    
    print("\nFinal Sorted Data:")
    print(final_sorted_df)
    
    
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total time for classical implementation = {total_time} seconds")

if __name__ == "__main__":
    classical_cluster_based_sort('phpB0xrNj.csv',sort_column="f3", n_clusters=20)

Original Data:
           f1      f2      f3      f4      f5      f6      f7      f8      f9  \
0    -0.4394 -0.0930  0.1718  0.4620  0.6226  0.4704  0.3578  0.0478 -0.1184   
1    -0.4348 -0.1198  0.2474  0.4036  0.5026  0.6328  0.4948  0.0338 -0.0520   
2    -0.2330  0.2124  0.5014  0.5222 -0.3422 -0.5840 -0.7168 -0.6342 -0.8614   
3    -0.3808 -0.0096  0.2602  0.2554 -0.4290 -0.6746 -0.6868 -0.6650 -0.8410   
4    -0.3412  0.0946  0.6082  0.6216 -0.1622 -0.3784 -0.4324 -0.4358 -0.4966   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
7331 -0.6842 -0.3280 -0.1984  0.2956  0.8786  0.8948  0.3118  0.1822  0.1012   
7332 -0.5912 -0.2420  0.8174  1.0000  0.4642  0.6428  0.6944  0.3056 -0.3888   
7333 -0.6696 -0.3730  0.1584  0.8910  1.0000  0.9762  0.9762  0.7684  0.4106   
7334 -0.5764 -0.1764  0.5106  0.3742 -0.1670 -0.5858 -0.7882 -0.7224 -0.6330   
7335 -0.6624 -0.3334  0.3666  0.4292 -0.2084 -0.5374 -0.4542 -0.6208 -0.6376   

         f10  ...    f6